In [ ]:
!pip install pandas networkx openpyxl


In [ ]:
import pandas as pd

# Đọc file Popular_Video
df = pd.read_excel('/content/Popular_Video_20250410_140539 (1).xlsx')

# Hiển thị 5 dòng đầu để kiểm tra
df.head()


,Video Title,Video Duration,Profile picture,Creator name,Region,Creator ID,Creator category,Followers,Published,Views,Likes,Comments,Engagement Rate,FastMoss Creator Detail,TikTok Creator Detail,TikTok Video Link,FastMoss Video Link
0,Ăn không lo Bé0 nhờ c iuuu @BÍCH NGỌC CENLY SH...,00:00:14,https://s.500fd.com/tt_author/101385208f6a7125...,Trần Hà Linh 🐰,VN,_halinhtran_,Beauty,1924072,2024-12-17 19:27,11860092,10597,589,0.09%,www.fastmoss.com/zh/influencer/detail/69999495...,https://www.tiktok.com/@_halinhtran_,https://www.tiktok.com/@_halinhtran_/video/744...,www.fastmoss.com/zh/media-source/video/7449340...
1,Muốn dáng đẹp thì tìm chị@Thảo Mộc Cenly - Thả...,00:00:15,https://s.500fd.com/tt_author/6584a6ebb35f9f74...,Suka🧸,VN,sukanenobita,Beauty,895750,2025-01-05 12:39,7200000,64700,753,0.92%,www.fastmoss.com/zh/influencer/detail/65571481...,https://www.tiktok.com/@sukanenobita,https://www.tiktok.com/@sukanenobita/video/745...,www.fastmoss.com/zh/media-source/video/7456286...
2,Muốn dáng đẹp đoán tết thì tìm chị@Thảo Mộc Ce...,00:00:14,https://s.500fd.com/tt_author/6584a6ebb35f9f74...,Suka🧸,VN,sukanenobita,Beauty,895750,2025-01-09 20:44,6300000,157500,1115,2.56%,www.fastmoss.com/zh/influencer/detail/65571481...,https://www.tiktok.com/@sukanenobita,https://www.tiktok.com/@sukanenobita/video/745...,www.fastmoss.com/zh/media-source/video/7457895...
3,"Dùng cenly đúng cách , săn sale siêu lớn",00:01:46,https://s.500fd.com/tt_author/c6180c9ebe34a89d...,Thảo Lê Cenly,VN,thaolecenly,Other,22561,2024-11-17 10:26,5565144,8690,519,0.19%,www.fastmoss.com/zh/influencer/detail/72797493...,https://www.tiktok.com/@thaolecenly,https://www.tiktok.com/@thaolecenly/video/7438...,www.fastmoss.com/zh/media-source/video/7438068...
4,Dáng đẹp tìm chị @Ánh Hồng Cenly ship ngay hộp...,00:00:15,https://s.500fd.com/tt_author/ad3075fdd4846e28...,𝓚𝓲𝓶 𝓐𝓷𝓱 🇻🇳,VN,taolakimanh_36,Beauty,1197413,2024-07-16 12:25,5368107,258917,485,4.86%,www.fastmoss.com/zh/influencer/detail/70122133...,https://www.tiktok.com/@taolakimanh_36,https://www.tiktok.com/@taolakimanh_36/video/7...,www.fastmoss.com/zh/media-source/video/7392085...


In [ ]:
import re

# Hàm trích tất cả hashtag trong một dòng văn bản
def extract_hashtags(text):
    if pd.isna(text): return []
    return re.findall(r"#\w+", text)

# Tạo cột mới chứa list hashtag từ tiêu đề
df["Hashtags"] = df["Video Title"].apply(extract_hashtags)

# Kiểm tra thử
df[["Creator ID", "Video Title", "Hashtags"]].head()


,Creator ID,Video Title,Hashtags
0,_halinhtran_,Ăn không lo Bé0 nhờ c iuuu @BÍCH NGỌC CENLY SH...,[#Cenlybichngoc]
1,sukanenobita,Muốn dáng đẹp thì tìm chị@Thảo Mộc Cenly - Thả...,[#suka]
2,sukanenobita,Muốn dáng đẹp đoán tết thì tìm chị@Thảo Mộc Ce...,[]
3,thaolecenly,"Dùng cenly đúng cách , săn sale siêu lớn",[]
4,taolakimanh_36,Dáng đẹp tìm chị @Ánh Hồng Cenly ship ngay hộp...,"[#xuhuong, #fyp]"


In [ ]:
import networkx as nx
from itertools import combinations

# Tạo graph vô hướng (vì hashtag không có hướng lan truyền rõ ràng)
G = nx.Graph()

# Duyệt từng dòng để gom creator theo hashtag
hashtag_to_creators = {}

for idx, row in df.iterrows():
    creator_id = row["Creator ID"]
    hashtags = row["Hashtags"]
    for tag in hashtags:
        if tag not in hashtag_to_creators:
            hashtag_to_creators[tag] = set()
        hashtag_to_creators[tag].add(creator_id)

# Tạo cạnh giữa các creator dùng chung hashtag
for tag, creators in hashtag_to_creators.items():
    for c1, c2 in combinations(creators, 2):
        if G.has_edge(c1, c2):
            G[c1][c2]["weight"] += 1
        else:
            G.add_edge(c1, c2, weight=1)

# Kết quả sơ bộ
print(f"Số nút: {G.number_of_nodes()}, Số cạnh: {G.number_of_edges()}")


Số nút: 245, Số cạnh: 13429


In [ ]:
nx.write_gexf(G, "creator_hashtag_network.gexf")
from google.colab import files
files.download("creator_hashtag_network.gexf")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import networkx as nx

G = nx.read_gexf("creator_hashtag_network.gexf")


#Metrics

In [ ]:
# Tính các chỉ số trung tâm
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G, normalized=True)
pagerank = nx.pagerank(G)
closeness_centrality = nx.closeness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G, max_iter=1000)

# Tạo bảng tổng hợp
df_metrics = pd.DataFrame({
    "Creator ID": list(degree_centrality.keys()),
    "Degree Centrality": list(degree_centrality.values()),
    "Betweenness Centrality": list(betweenness_centrality.values()),
    "Closeness Centrality": list(closeness_centrality.values()),
    "Eigenvector Centrality": list(eigenvector_centrality.values()),
    "PageRank": list(pagerank.values())
})

# Hiển thị bảng
df_metrics.head(10)


,Creator ID,Degree Centrality,Betweenness Centrality,Closeness Centrality,Eigenvector Centrality,PageRank
0,cenlyvietnam.vn,0.655738,0.004680,0.739394,0.087109,0.007641
1,tuyencenly1,0.627049,0.001172,0.726190,0.086601,0.005618
2,taolakimanh_36,0.704918,0.015022,0.767296,0.071681,0.005872
3,odayne05,0.622951,0.001093,0.721893,0.086238,0.005195
4,nhutrucbeuaty09,0.659836,0.002202,0.741641,0.088019,0.006982
5,congtycenlyvietnam2,0.709016,0.009987,0.772152,0.089278,0.010354
6,kieuduyencenly77,0.692623,0.004667,0.760125,0.089009,0.010252
7,tramhuong.cenly,0.635246,0.001032,0.728358,0.087145,0.005834
8,nanaliiu.2107,0.643443,0.001440,0.732733,0.087388,0.006896
9,shopnhahuyen03,0.639344,0.001575,0.732733,0.087166,0.006117


In [ ]:
df_metrics.to_csv("creator_network_metrics_full.csv", index=False)

In [ ]:
# Tạo bảng thống kê mô tả cho các chỉ số mạng
summary_stats = df_metrics.describe().T

# Lấy các thống kê cơ bản nhất
summary_stats = summary_stats[["mean", "std", "min", "50%", "max"]]
summary_stats.columns = ["Trung bình", "Độ lệch chuẩn", "Giá trị nhỏ nhất", "Trung vị", "Giá trị lớn nhất"]

# Hiển thị bảng
summary_stats


,Trung bình,Độ lệch chuẩn,Giá trị nhỏ nhất,Trung vị,Giá trị lớn nhất
Degree Centrality,0.449281,0.213413,0.004098,0.512295,0.860656
Betweenness Centrality,0.002433,0.004981,0.000000,0.000506,0.034844
Closeness Centrality,0.645241,0.098443,0.324468,0.668493,0.874552
Eigenvector Centrality,0.054758,0.032979,0.000107,0.074638,0.093564
PageRank,0.004082,0.002275,0.000635,0.003469,0.011039


In [ ]:
# Assortativity Coefficient
assortativity = nx.degree_assortativity_coefficient(G)

# Average Shortest Path Length và Diameter
if nx.is_connected(G.to_undirected()):
    avg_path = nx.average_shortest_path_length(G)
    diameter = nx.diameter(G)
else:
    avg_path = "Graph is not connected"
    diameter = "Graph is not connected"

# Clustering Coefficient
clustering = nx.average_clustering(G)

# Hiển thị kết quả
print("Assortativity Coefficient:", assortativity)
print("Average Shortest Path Length:", avg_path)
print("Diameter:", diameter)
print("Average Clustering Coefficient:", clustering)


Assortativity Coefficient: 0.20530847215103018
Average Shortest Path Length: 1.5912010705921713
Diameter: 5
Average Clustering Coefficient: 0.8850070753798152
